<a href="https://colab.research.google.com/github/lanxaustralis/NLP-Projects/blob/master/S2S_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt update -q
!apt install cuda gcc-6 g++-6 -y -q
!ln -s /usr/bin/gcc-6 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-6 /usr/local/cuda/bin/g++
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.2/julia-1.2.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; add Knet; precompile"'
# Restart with new IPython.
import os
import signal
os.kill(os.getpid(), signal.SIGTERM)

--2019-11-04 19:01:40--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?T2taPXUGfsB-9whnBSrXdsTP8yjb_SuMfh-PDCMH8V0QEWOYRRl4tDIByTtKerrKsFJPCBCLv7z_4mcgezfxz6QQo3G0uAXtw4JzsGbIuNtGrd5ku76eFLs76Giq-DyPKSDD3XJWTsVyUxGnREJtHBvMnx9XlWV5uuWWKC90b08aImcr-1WnBo9b67SHGIfSn8cNuklYC4_yD6qtxRU0 [following]
--2019-11-04 19:01:41--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?T2taPXUGfsB-9whnBSrXdsTP8yjb_SuMfh-PDCMH8V0QEWOYRRl4tDIByTtKerrKsFJPCBCLv7z_4mcgezfxz6QQo3G0uAXtw4JzsGbIuNtGrd

### After executing the first cell click on `connect to hosted runtime` which you can find in the drop menu on the upper right corner on the left of Editing

In [1]:
import Pkg
Pkg.add("IterTools")
Pkg.add("AutoGrad")
using Pkg
using Knet, Test, Base.Iterators, IterTools, Random # , LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, use with Float64

macro size(z, s) # for debugging
    esc(:(@assert (size($z) == $s) string(summary($z), !=, $s))) # for debugging
end # for debugging

Pkg.add("CuArrays")
Pkg.build("CuArrays")

using CuArrays: CuArrays, usage_limit

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed IterTools ─ v1.2.0
  Updating `~/.julia/environments/v1.2/Project.toml`
  [c8e1da08] + IterTools v1.2.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [c8e1da08] + IterTools v1.2.0
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [6710c13c] + AutoGrad v1.2.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


┌ Info: Precompiling IterTools [c8e1da08-722c-5040-9ed9-7db0dc04731e]
└ @ Base loading.jl:1242


 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [3a865a2d] + CuArrays v1.3.0
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
  Building Conda → `~/.julia/packages/Conda/kLXeC/deps/build.log`
  Building FFTW ─→ `~/.julia/packages/FFTW/MJ7kl/deps/build.log`


In [2]:
Pkg.add("Statistics")
import Statistics
using Statistics

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [10745b16] + Statistics 
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


### After setting up the environment, check what we have

In [3]:
Pkg.update()
pkgs = Pkg.installed()

for package in keys(pkgs)
    if pkgs[package] == nothing
        pkgs[package] = VersionNumber("0.0.1")
    end
    println("Package name: ", package, " Version: ", pkgs[package])
end

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed CUDAdrv ──── v4.0.1
 Installed CUDAnative ─ v2.5.0
 Installed CuArrays ─── v1.4.0
 Installed Knet ─────── v1.2.7
  Updating `~/.julia/environments/v1.2/Project.toml`
  [3a865a2d] ↑ CuArrays v1.3.0 ⇒ v1.4.0
  [1902f260] ↓ Knet v1.3.0 ⇒ v1.2.7
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [c5f51814] ↑ CUDAdrv v3.1.0 ⇒ v4.0.1
  [be33ccc6] ↑ CUDAnative v2.4.0 ⇒ v2.5.0
  [3a865a2d] ↑ CuArrays v1.3.0 ⇒ v1.4.0
  [1902f260] ↓ Knet v1.3.0 ⇒ v1.2.7
  Building Knet → `~/.julia/packages/Knet/IIjk8/deps/build.log`
Package name: AutoGrad Version: 1.2.0
Package name: IterTools Version: 1.2.0
Package name: Statistics Version: 0.0.1
Package name: Knet Version: 1.2.7
Package name: CuArrays Version: 1.4.0
Package name: IJulia Version: 1.20.0


### Set some constants

In [4]:
#CuArrays.usage_limit[] = 9_000_000_000
#array_type = KnetArray # For GPU instances
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
BATCH_SIZE = 64

64

### Part -1: Copy what we had



In [6]:
struct Vocab
    w2i::Dict{String,Int}
    i2w::Vector{String}
    unk::Int
    eos::Int
    tokenizer
end

function Vocab(
    file::String;
    tokenizer = split,
    vocabsize = Inf,
    mincount = 1,
    unk = "<unk>",
    eos = "<s>",
)
    vocab_freq = Dict{String,Int64}(unk => 1, eos => 1)
    w2i = Dict{String,Int64}(unk => 2, eos => 1)
    i2w = Vector{String}()

    push!(i2w, eos)
    push!(i2w, unk)

    open(file) do f
        for line in eachline(f)
            sentence = strip(lowercase(line))
            sentence = tokenizer(line, [' '], keepempty = false)

            for word in sentence
                word == unk && continue
                word == eos && continue # They are default ones to be added later
                vocab_freq[word] = get!(vocab_freq, word, 0) + 1
            end
        end
        close(f)
    end


    # End of vanilla implementation of the vocaulary
    # From here we must add the mincount and vocabsize properties
    # We must change the first two property of the vocab wrt those paramaters
    vocab_freq = sort!(
        collect(vocab_freq),
        by = tuple -> last(tuple),
        rev = true,
    )

    if length(vocab_freq) > vocabsize - 2 # eos and unk ones
        vocab_freq = vocab_freq[1:vocabsize-2] # trim to fit the size
    end

    #vocab_freq = reverse(vocab_freq)

    while true
        length(vocab_freq) == 0 && break
        word, freq = vocab_freq[end]
        freq >= mincount && break # since it is already ordered
        vocab_freq = vocab_freq[1:(end-1)]
    end
    #pushfirst!(vocab_freq,unk=>1,eos=>1) # freq does not matter, just adding the
    for i = 1:length(vocab_freq)
        word, freq = vocab_freq[i]
        ind = (get!(w2i, word, 1 + length(w2i)))
        (length(i2w) < ind) && push!(i2w, word)
    end

    return Vocab(w2i, i2w, 2, 1, tokenizer)
end

struct TextReader
    file::String
    vocab::Vocab
end

word2ind(dict, x) = get(dict, x, 2)  # Unk as 2

#Implementing the iterate function
function Base.iterate(r::TextReader, s = nothing)
    if s == nothing
        state = open(r.file)
        Base.iterate(r, state)
    else
        if eof(s) == true
            close(s)
            return nothing
        else
            line = readline(s)
            line = strip(lowercase(line))
            sent = r.vocab.tokenizer(line, [' '], keepempty = false)
            sent_ind = Int[]
            for word in sent
                ind = word2ind(r.vocab.w2i, word)
                push!(sent_ind, ind)
            end
            return (sent_ind, s)
        end
    end
end


Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

#Embed
struct Embed
    w
end

function Embed(vocabsize::Int, embedsize::Int)
    Embed(param(embedsize, vocabsize))
end

function (l::Embed)(x)
    l.w[:, x]
end

#Linear
struct Linear
    w
    b
end

function Linear(inputsize::Int, outputsize::Int)
    Linear(
        param(outputsize, inputsize),
        param0(outputsize),
    )
end

function (l::Linear)(x)
    l.w * mat(x, dims = 1) .+ l.b
end

# Mask!
function mask!(a, pad)
    matr = a
    for j = 1:size(matr)[1]
        i = 0
        while (i < length(matr[j, :]) - 1)
            if matr[j, length(matr[j, :])-i-1] != pad
                break

            elseif matr[j, length(matr[j, :])-i] == pad
                matr[j, length(matr[j, :])-i] = 0
            end
            i += 1
        end
    end
    return matr
end

mask! (generic function with 1 method)

### Part 0: Load Data

In [7]:
datadir = "datasets/tr_to_en"

if !isdir(datadir)
    download(
        "http://www.phontron.com/data/qi18naacl-dataset.tar.gz",
        "qi18naacl-dataset.tar.gz",
    )
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end

if !isdefined(Main, :tr_vocab)
    tr_vocab = Vocab("$datadir/tr.train", mincount = 5)
    en_vocab = Vocab("$datadir/en.train", mincount = 5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    @info "Testing data"
    @test length(tr_vocab.i2w) == 38126
    @test length(first(tr_test)) == 16
    @test length(collect(tr_test)) == 5029
end

┌ Info: Testing data
└ @ Main In[7]:20


Test Passed

### Part 1 - Minibatching

In [8]:
struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(
    src::TextReader,
    tgt::TextReader;
    batchmaker = arraybatch,
    batchsize = BATCH_SIZE,
    maxlength = typemax(Int),
    batchmajor = false,
    bucketwidth = 10,
    numbuckets = min(BATCH_SIZE, maxlength ÷ bucketwidth),
)
    buckets = [[] for i = 1:numbuckets] # buckets[i] is an array of sentence pairs with similar length
    MTData(
        src,
        tgt,
        batchsize,
        maxlength,
        batchmajor,
        bucketwidth,
        buckets,
        batchmaker,
    )
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

function Base.iterate(d::MTData, state = nothing)
    if state == nothing
        for b in d.buckets
            empty!(b)
        end
        state_src, state_tgt = nothing, nothing
    else
        state_src, state_tgt = state
    end
    bucket, ibucket = nothing, nothing


    while true
        iter_src = (state_src === nothing ? iterate(d.src) :
                    iterate(d.src, state_src))
        iter_tgt = (state_tgt === nothing ? iterate(d.tgt) :
                    iterate(d.tgt, state_tgt))

        if iter_src === nothing
            ibucket = findfirst(x -> !isempty(x), d.buckets)
            bucket = (ibucket === nothing ? nothing : d.buckets[ibucket])
            break
        else
            sent_src, state_src = iter_src
            sent_tgt, state_tgt = iter_tgt
            if length(sent_src) > d.maxlength || length(sent_src) == 0
                continue
            end
            ibucket = min(
                1 + (length(sent_src) - 1) ÷ d.bucketwidth,
                length(d.buckets),
            )
            bucket = d.buckets[ibucket]
            push!(bucket, (sent_src, sent_tgt))
            if length(bucket) === d.batchsize
                break
            end
        end
    end
    if bucket === nothing
        return nothing
    end

    batch = d.batchmaker(d, bucket)

    empty!(bucket)
    return batch, (state_src, state_tgt)
end


function arraybatch(d::MTData, bucket)
    bucketx = map(x -> x[1], bucket)
    buckety = map(x -> x[2], bucket)
    batch_x = fill(d.src.vocab.eos, length(bucketx), maximum(length.(bucketx)))
    for i = 1:length(bucket)
        batch_x[i, end-length(bucketx[i])+1:end] = bucketx[i]
    end
    batch_y = fill(
        d.tgt.vocab.eos,
        length(buckety),
        maximum(length.(buckety)) + 2,
    )
    for i = 1:length(bucket)
        batch_y[i, 2:length(buckety[i])+1] = buckety[i]
    end

    return (batch_x, batch_y)
end

#-

@info "Testing MTData"
dtrn = MTData(tr_train, en_train)
ddev = MTData(tr_dev, en_dev)
dtst = MTData(tr_test, en_test)

x, y = first(dtst)

#@test length(collect(dtst)) == 48
#@test size.((x, y)) == ((128, 10), (128, 24))
@test x[1, 1] == tr_vocab.eos
@test x[1, end] != tr_vocab.eos
@test y[1, 1] == en_vocab.eos
@test y[1, 2] != en_vocab.eos
@test y[1, end] == en_vocab.eos

┌ Info: Testing MTData
└ @ Main In[8]:110


Test Passed

## Part 2 - S2S w/o attention

In [9]:
struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
    srcvocab::Vocab     # source language vocabulary
    tgtvocab::Vocab     # target language vocabulary
end

function S2S_v1(
    hidden::Int,         # hidden size for both the encoder and decoder RNN
    srcembsz::Int,       # embedding size for source language
    tgtembsz::Int,       # embedding size for target language
    srcvocab::Vocab,     # vocabulary for source language
    tgtvocab::Vocab;     # vocabulary for target language
    layers = 1,            # number of layers
    bidirectional = false, # whether encoder RNN is bidirectional
    dropout = 0,
)           # dropout probability


    layerMultiplier = bidirectional ? 2 : 1

    S2S_v1(
        Embed(length(srcvocab.i2w), srcembsz),
        RNN(
            srcembsz,
            hidden,
            numLayers = layers,
            bidirectional = bidirectional,
            dropout = dropout
        ),
        Embed(length(tgtvocab.i2w), tgtembsz),
        RNN(
            tgtembsz,
            hidden,
            numLayers = layerMultiplier * layers,
            dropout = dropout
        ),
        Linear(hidden, length(tgtvocab.i2w)),
        dropout,
        srcvocab,
        tgtvocab,
    )

end

function (s::S2S_v1)(src, tgt; average = true)
    #B,Tx = size(src,2)
    B, Ty = size(tgt)
    Ty -= 1 # Crop one
    # Ex, Ey = length(model.srcembed([1])), length(model.tgtembed([1]))

    rnn_encoder = s.encoder
    rnn_decoder = s.decoder
    project = s.projection

    # Lx, Ly = rnn_encoder.numLayers, rnn_decoder.numLayers
    # Hx, Hy = rnn_encoder.hiddenSize, rnn_decoder.hiddenSize
    # Dx = Ly/Lx

    emb_out_src = s.srcembed(src)
    #@test size(emb_out_src)== (Ex,B,Tx) # Done

    # Safe for repetitive usage
    rnn_encoder.h = 0
    rnn_encoder.c = 0

    y_enc = rnn_encoder(dropout(emb_out_src,s.dropout))
    #@test size(y_enc) == (Hx*Dx,B,Tx)
    h_enc = rnn_encoder.h
    #@test size(h_enc) == (Hx,B,Lx*Dx)
    c_enc = rnn_encoder.c

    emb_out_tgt = s.tgtembed(tgt[:, 1:end-1])
    #@test size(emb_out_tgt)== (Ey,B,Ty)

    rnn_decoder.h = h_enc
    rnn_decoder.c = c_enc
    y_dec = rnn_decoder(emb_out_tgt)
    #@test size(y_dec)==(Hy,B,Ty)

    project_inp = reshape(y_dec, :, B * Ty)
    project_out = project(project_inp)

    #@test size(project_out)==(length(project.b),B*Ty)

    verify = deepcopy(tgt)
    mask!(verify, s.tgtvocab.eos)

    average && return mean(nll(project_out, verify[:, 2:end]))
    return nll(project_out, verify[:, 2:end]; average = false)
end

#-

@info "Testing S2S_v1"
Knet.seed!(1)
model = S2S_v1(
    512,
    512,
    512,
    tr_vocab,
    en_vocab;
    layers = 2,
    bidirectional = true,
    dropout = 0.2,
)
GC.gc();(x, y) = first(dtst)
## Your loss can be slightly different due to different ordering of words in the vocabulary.
## The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
#@test model(x,y; average=false) == (14097.471f0, 1432)  !!!!!!
GC.gc();println(model(x, y; average = false))

┌ Info: Testing S2S_v1
└ @ Main In[9]:99


(14471.718f0, 1470)


### Loss Function

In [10]:
function loss(model, data; average = true)
    total_loss = 0
    total_word = 0

    for (x, y) in collect(data)
        curr_loss, curr_word = model(x, y; average = false)
        total_loss += curr_loss
        total_word += curr_word
    end
    if total_word == 0
      average && return 0
      return 0,0
    end
    average && return total_loss / total_word
    return (total_loss, total_word)

end

#-

@info "Testing loss"
@time res = loss(model, dtst, average = false)
println(res)
#@test res == (1.0429117f6, 105937) !!!!!!!!!!

┌ Info: Testing loss
└ @ Main In[10]:21


  2.540305 seconds (1.97 M allocations: 193.435 MiB, 11.60% gc time)
(1.04291994f6, 105937)


### Train Function

In [11]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev;average=true)
    progress!(adam(model, trn), steps = 100) do y
        losses = [loss(model, d) for d in (dev, tst...)]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

train! (generic function with 1 method)

### Train Parameters

In [12]:
@info "Training S2S_v1"
epochs = 10
ctrn = collect(dtrn)
trnx10 = collect(flatten(shuffle!(ctrn) for i = 1:epochs))
trn20 = ctrn[1:20]
dev38 = collect(ddev)

┌ Info: Training S2S_v1
└ @ Main In[12]:1


69-element Array{Tuple{T,T} where T,1}:
 ([1 1 … 2 3; 1 1 … 2 3; … ; 1 1 … 553 3; 1 9 … 8 8], [1 43 … 1 1; 1 37 … 1 1; … ; 1 43 … 1 1; 1 23 … 1 1])                
 ([1000 1673 … 1066 3; 1 1 … 25 3; … ; 1 1 … 130 3; 1 7 … 2 3], [1 60 … 4 1; 1 9 … 1 1; … ; 1 11 … 1 1; 1 19 … 1 1])       
 ([1 1 … 2865 3; 1 1 … 2 3; … ; 1 1 … 2 3; 1 1 … 1618 3], [1 14 … 1 1; 1 158 … 1 1; … ; 1 29 … 1 1; 1 11 … 4 1])           
 ([1 1 … 827 3; 1 1 … 2 3; … ; 1 296 … 442 3; 1 1 … 2529 24], [1 20 … 1 1; 1 9 … 1 1; … ; 1 775 … 1 1; 1 41 … 1 1])        
 ([6 35117 … 1494 3; 43 15 … 10233 3; … ; 1 1 … 2528 3; 1 1 … 21 3], [1 6 … 1 1; 1 20 … 1 1; … ; 1 21 … 1 1; 1 21 … 1 1])  
 ([1 1 … 8575 3; 1 1 … 2 3; … ; 1 1 … 2785 3; 1 7 … 2329 3], [1 21 … 1 1; 1 13 … 1 1; … ; 1 6 … 1 1; 1 6 … 1 1])           
 ([1 54 … 630 3; 1 1 … 84 3; … ; 9 9 … 3 8; 1 1 … 587 3], [1 11 … 1 1; 1 37 … 1 1; … ; 1 23 … 1 1; 1 27 … 1 1])            
 ([1 1 … 2 3; 1 1 … 95 14; … ; 1 1 … 2 3; 1 1 … 337 3], [1 6 … 1 1; 1 75 … 1 1; … ; 1 6 … 1 

### Training

In [17]:
GC.gc();
model = train!(model, trnx10, dev38, trn20)

┣████████████████████┫ [100.00%, 28580/28580, 01:33:59/01:33:59, 5.07i/s] (3.4938595f0, 2.0767374f0)


S2S_v1(Embed(P(KnetArray{Float32,2}(512,38126))), LSTM(input=512,hidden=512,bidirectional,layers=2,dropout=0.2), Embed(P(KnetArray{Float32,2}(512,18857))), LSTM(input=512,hidden=512,layers=4,dropout=0.2), Linear(P(KnetArray{Float32,2}(18857,512)), P(KnetArray{Float32,1}(18857))), 0.2, Vocab(Dict("ağacından" => 35370,"komuta" => 13566,"ellisi" => 25239,"adresini" => 22820,"yüzeyi" => 4051,"paris'te" => 9494,"kafamdaki" => 18790,"yüzeyinde" => 5042,"geçerlidir" => 6612,"kökten" => 7774…), ["<s>", "<unk>", ".", ",", "bir", "ve", "bu", "''", "``", "için"  …  "seçmemiz", "destekleyip", "karşılaştırılabilir", "ördeğin", "gününüzü", "bağışçı", "istismara", "yaşça", "tedci", "fakültesi'nde"], 2, 1, split), Vocab(Dict("middle-income" => 13398,"photosynthesis" => 7689,"polarizing" => 17881,"henry" => 4248,"abducted" => 15691,"rises" => 6225,"hampshire" => 13888,"whiz" => 16835,"cost-benefit" => 13137,"progression" => 5549…), ["<s>", "<unk>", ",", ".", "the", "and", "to", "of", "a", "that"  …  "a

### Save & Load

In [30]:
Knet.save("s2s_v1.jld2","model",model)
model = Knet.load("s2s_v1.jld2","model")

S2S_v1(Embed(P(KnetArray{Float32,2}(512,38126))), LSTM(input=512,hidden=512,bidirectional,layers=2,dropout=0.2), Embed(P(KnetArray{Float32,2}(512,18857))), LSTM(input=512,hidden=512,layers=4,dropout=0.2), Linear(P(KnetArray{Float32,2}(18857,512)), P(KnetArray{Float32,1}(18857))), 0.2, Vocab(Dict("dev" => 1277,"komuta" => 13566,"ellisi" => 25239,"adresini" => 22820,"yüzeyi" => 4051,"paris'te" => 9494,"kafamdaki" => 18790,"yüzeyinde" => 5042,"geçerlidir" => 6612,"kökten" => 7774…), ["<s>", "<unk>", ".", ",", "bir", "ve", "bu", "''", "``", "için"  …  "seçmemiz", "destekleyip", "karşılaştırılabilir", "ördeğin", "gününüzü", "bağışçı", "istismara", "yaşça", "tedci", "fakültesi'nde"], 2, 1, split), Vocab(Dict("middle-income" => 13398,"photosynthesis" => 7689,"polarizing" => 17881,"henry" => 4248,"abducted" => 15691,"rises" => 6225,"hampshire" => 13888,"whiz" => 16835,"cost-benefit" => 13137,"progression" => 5549…), ["<s>", "<unk>", ",", ".", "the", "and", "to", "of", "a", "that"  …  "archaea"

### Translation

In [31]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    # Preperation for initial step
    B,Tx = size(src)
    max_step = stopfactor * Tx
    tgt_eos = s.tgtvocab.eos

    tgt = fill(tgt_eos, (B, 1)) # size as (B,2)
    #output = fill(tgt_eos,(B,max_step))
    output = Array{Int64}(undef, B, max_step)

    rnn_encoder = s.encoder
    rnn_decoder = s.decoder
    project = s.projection

    emb_out_src = s.srcembed(src)

    # Safe for repetitive usage
    rnn_encoder.h = 0
    rnn_encoder.c = 0

    y_enc = rnn_encoder(emb_out_src)
    rnn_decoder.h = rnn_encoder.h
    rnn_decoder.c = rnn_encoder.c

    step = 0
    #@test Ty == size(tgt,2)

    while step < max_step
        step += 1
        emb_out_tgt = s.tgtembed(tgt)

        y_dec = rnn_decoder(emb_out_tgt)

        project_inp = reshape(y_dec, :, B)
        project_out = project(project_inp)

        eos_num = 0
        for i = 1:B
            # Assigns the position of the highest token
            col = project_out[:, i]
            colMax = col[1]
            index = 1
            for j in 1:length(col)
                if colMax<col[j]
                    colMax, index = col[j],j
                end
            end
            if index == tgt_eos
                eos_num += 1
            end
            tgt[i] = index
        end

        output[:,step] = tgt
        eos_num == B && break # all produced eos


    end

    return output[:, 1:step]

end


## Utility to convert int arrays to sentence strings
function int2str(y, vocab)
    y = vec(y)
    ysos = findnext(w -> !isequal(w, vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1 + length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

int2str (generic function with 1 method)

### Generate Translations

In [33]:
@info "Generating some translations"
d = MTData(tr_dev, en_dev, batchsize = 1) |> collect
(src, tgt) = rand(d)
out = model(src)
println("SRC: ", int2str(src, model.srcvocab))
println("REF: ", int2str(tgt, model.tgtvocab))
println("OUT: ", int2str(out, model.tgtvocab))

SRC: <unk> ilgilenmek yapabileceğiniz en tedavi edici ve meydan okuyucu davranış .
REF: gardening is the most therapeutic and <unk> act you can do .
OUT: the best way to do is to change the most effective and <unk> and <unk> .


┌ Info: Generating some translations
└ @ Main In[33]:1


### Define BLEU

In [34]:
CuArrays.usage_limit[] = 10_000_000_000

function bleu(s2s, d::MTData)
    d = MTData(d.src, d.tgt, batchsize = 1)
    reffile = d.tgt.file
    hypfile, hyp = mktemp()
    for (x, y) in progress(collect(d))
        g = s2s(x)
        for i = 1:size(y, 1)
            println(hyp, int2str(g[i, :], d.tgt.vocab))
        end
    end
    close(hyp)
    isfile("multi-bleu.perl") || download(
        "https://github.com/moses-smt/mosesdecoder/raw/master/scripts/generic/multi-bleu.perl",
        "multi-bleu.perl",
    )
    run(pipeline(`cat $hypfile`, `perl multi-bleu.perl $reffile`))
    return hypfile
end

# Calculating dev BLEU takes about 45 secs on a V100. We get about 8.0 BLEU which is pretty
# low. As can be seen from the sample translations a loss of ~3+ (perplexity ~20+) or a BLEU
# of ~8 is not sufficient to generate meaningful translations.



bleu (generic function with 1 method)

### Calculate BLEU

In [35]:
@info "Calculating BLEU"
GC.gc();bleu(model, ddev)

┌ Info: Calculating BLEU
└ @ Main In[35]:1
┣                    ┫ [0.35%, 14/4045, 00:59/04:42:21, 5.12s/i] 

InterruptException: ignored